### Our testing framework is coming along!

Now we have a relatively functional test runner to go along with our matchers! Let's import our progress so far on our test suite:

In [ ]:
from phoenix_test.matchers import FailedAssertion, Assertion, assert_that
from phoenix_test.test import Test

Look, I'll be honest: I'm getting pretty tired of implementing a crappy `find_twos` in each of these lessons. So for now, we're gonna graduate to pretending we got promoted to Test Engineer for the Python Programming Language, responsible for writing tests to evaluate the Python builtins themselves. I'm sure the Python Software Foundation put our checks in the mail today.

In [ ]:
class SortedTests(Test):
    def test_sort_integers(self):
        assert_that(sorted([3, 1, 2])).equals([1, 2, 3])

    def test_sort_strings(self):
        assert_that(sorted(["C", "A", "B"])).equals(["A", "B", "C"])

class DirTests(Test):
    def test_dir_any_object(self):
        assert_that(dir(object)).has_items('__init__')

    def test_dir_custom_object(self):
        class NewClass():
            some_attribute = "Hello!"
        assert_that(dir(NewClass)).has_items('some_attribute')

Test.run_all()

Suppose we have the following code that represents a scientific experiment. We want to be able to sort our subjects into control and experimental groups for running the trial:

In [ ]:
import random 

class Experiment():
    def __init__(self, name):
        self.name = name
        self.subjects = []

class Subject:
    pass
        
class Trial():
    def __init__(self, 
                 control, 
                 experimental, 
                 experiment_proportion = 0.3, 
                 subjects=[]):
        self.control = control
        self.experimental = experimental
        self.experiment_proportion = experiment_proportion
        self.subjects = subjects
    
    def sort_subjects(self):
        for subject in self.subjects:
            if random.random() < self.experiment_proportion:
                self.experimental.subjects.append(subject)
            else:
                self.control.subjects.append(subject)

Of course, experiments are serious business, so we want to write some tests to make sure that our `Trial` is doing what we think it's doing:

In [ ]:
class TrialTests(Test):
    
    def setup(self):
        experimental = Experiment("Candy for breakfast")
        control = Experiment("A normal, sensible breakfast")
        subjects = [Subject() for i in range(10)]
        self.trial = Trial(
            control=control, 
            experimental=experimental,
            subjects=subjects
        )
    
    def test_default(self):
        assert_that(self.trial.subjects).has_size(10)
        self.trial.sort_subjects()
        print(f"Control group size: {len(self.trial.control.subjects)}")
        print(f"Experimental group size: {len(self.trial.experimental.subjects)}")

I have done something unusual here: I am printing out some values from the test to demonstrate how the subjects are split into their control and experimental groups. Run the line of code below a few times. What do you notice?

In [ ]:
TrialTests().run()

If you run the above several times, you'll notice that the control group size and the experimental group size are different from run to run. We call code like this **non-deterministic.** We're using random sorting ensure that our control and experimental group sorting isn't biased, but that means that we cannot check exactly what this code is doing as written.

Luckily, we have some options for situations like this. The most common one, especially in an object-oriented language, is **dependency injection**. Here's how that looks:

In [ ]:
class TrialWithDI(Trial):
    def __init__(self, 
                 control, 
                 experimental, 
                 sorting_function,
                 subjects=[]):
        super().__init__(control,experimental,0.3,subjects)
        self.sorting_function = sorting_function
        
    def sort_subjects(self):
        for subject in self.subjects:
            if self.sorting_function():
                self.experimental.subjects.append(subject)
            else:
                self.control.subjects.append(subject)

In this case, the `sort_subjects` function is a _dependency_ upon which `ExperimentWithDI` depends. Sometimes dependency injection is considered an implementation of an idea called **inversion of control**, that is:

1. The "traditional" direction of control in an object-oriented language is that superclasses pass behavior "down" to subclasses
2. In dependency injection, dependencies pass behavior "up" into the classes that accept them as dependency attributes

Now, part of the above isn't great, which is what we're doing to the `experiment_proportion`. The subclass doesn't accept it in the initializer, instead calling the superclass initializer with a throwaway value. The subclass then overrides `sort_subjects` to _not use_ experiment_proportion at all. It's not the _worst_ possible offender of removing behavior in subclasses, but it's weird and confusing. This is the kind of place where I might put a comment in some code to explain why I'm doing this.

### Challenge: 

Add a comment to the `ExperimentWithDI` code to explain why we do not need `experiment_proportion` anymore.

### Okay, so anyway.

Here's how we could use dependency injection to surgically remove the randomness of the `Trial` class for testing:

In [ ]:
class TrialTests(Test):
    
    def setup(self):
        experimental = Experiment("Candy for breakfast")
        control = Experiment("A normal, sensible breakfast")
        subjects = [Subject() for i in range(10)]
        
        self.trial = TrialWithDI(
            control=control, 
            experimental=experimental,
            sorting_function=lambda: True,
            subjects=subjects
        )
    
    def test_default(self):
        assert_that(self.trial.subjects).has_size(10)
        self.trial.sort_subjects()
        print(f"Control group size: {len(self.trial.control.subjects)}")
        print(f"Experimental group size: {len(self.trial.experimental.subjects)}")

In [ ]:
TrialTests().run()

This sorting function always returns true, so all of the subjects get sorted into the `experimental` group. We can use a test like this to evaluate all the _other_ mechaics of our `TrialWithDI` class, then pass in `random.random() < 0.3` for the _real_ implementation to get the random effect we need.

## Now let's try something wild.

Can we write a `mock` decorator to handle injecting some mock behavior for us, without reimplementing the `Trial` class?

🚨🚨The following code was written by a professional developer on a closed course. Do not try this at home.🚨🚨

![](../images/car_drift.png)

In [ ]:
def mock(call, to_return):
    def decorator(func):
        def wrapper(testobject):
            module = eval('.'.join(call.split('.')[:-1]))
            module_function_name = call.split('.')[-1]
            module_function_body = getattr(module, module_function_name)
            setattr(module, module_function_name, lambda: to_return)
            
            func(testobject)
            
            setattr(module, module_function_name, module_function_body)
        return wrapper
    return decorator

We'd use it like this:

In [ ]:
class TrialTests(Test):
    
    def setup(self):
        experimental = Experiment("Candy for breakfast")
        control = Experiment("A normal, sensible breakfast")
        subjects = [Subject() for i in range(10)]
        self.trial = Trial(
            control=control, 
            experimental=experimental,
            subjects=subjects
        )
    
    @mock(call='random.random', to_return=0.1)
    def test_default(self):
        assert_that(self.trial.subjects).has_size(10)
        self.trial.sort_subjects()
        print(f"Control group size: {len(self.trial.control.subjects)}")
        print(f"Experimental group size: {len(self.trial.experimental.subjects)}")

In [ ]:
TrialTests().run()

OK. So. Let's evaluate this code. But before we can do that, let's make sure we understand what this code is doing.

### Challenge:

1. Go through the `mock` decorator, line by line, and figure out what it is doing. 

You have seen this function before:

- `getattr`

You are likely to need to look up:

- `eval`
- `setattr`

2. What is going to happen if we don't include `setattr(module, module_function_name, module_function_body)` in the mock method? If you need a hint, you can comment out that line, run the mock method, run the blocks of code that use it, and then run the line below:

In [ ]:
random.random()

3. Does doing this with the code seem like a good idea to you? What are the benefits and risks of what we've done here?